In [1]:
%pip install flair

In [2]:
%pip install transformers

In [3]:
import os
import re
import numpy as np 
import pandas as pd
import random
import sklearn
from sklearn.metrics import accuracy_score

import transformers
from transformers import BertTokenizer, BertModel

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
device = 'cuda' if cuda.is_available() else 'cpu'

!pip3 install pickle5
import pickle5 as pickle

device

'cpu'

In [4]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from textblob import TextBlob
import torch
from flair.embeddings import BertEmbeddings
from flair.data import Sentence
import numpy as np
from collections import Counter
import seaborn as sn
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import textblob
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support
import re

BERT = BertEmbeddings()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a Bert

### BERT fine-tuning for document classification

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
with open('/content/drive/My Drive/SLIM/LREC/cozmo_filtered_data.pkl', "rb") as fh:
  cozmo_data = pickle.load(fh)
with open('/content/drive/My Drive/SLIM/LREC/misty_filtered_data.pkl', "rb") as fh:
  misty_data = pickle.load(fh)

In [7]:
cozmo_data

,Timestamp,Description,Interst/Alarm,Confusion/Understanding,Frusteration/Relief,Sorrow/Joy,Anger/Gratitude,Fear/Hope,Boredom/Surprise,Disgust/Desire
0,1.617917e+09,It went to investigate a thing it saw,1,1,3,3,3,4,4,5
1,1.619714e+09,drove forward for a bit and then turned to his...,2,3,3,3,3,3,3,3
2,1.619714e+09,"just drove forward, made a little sound at the...",2,3,3,3,3,3,3,3
4,1.619714e+09,looked down squinted eyes and then looked forw...,2,2,3,3,3,3,3,3
5,1.619714e+09,"lifted front part up, moved forward and turned...",4,3,3,3,2,3,4,3
...,...,...,...,...,...,...,...,...,...,...
648,1.637355e+09,At first the robot tilted thier head up and sa...,1,2,3,1,3,3,3,3
649,1.637355e+09,The robot first moved straight with a slight r...,1,2,1,3,1,2,3,1
650,1.637355e+09,"At first, the robot raised their head and hand...",1,5,5,5,5,5,5,3
651,1.637355e+09,"The robot turned around nearly 180 degrees, lo...",3,4,3,1,3,1,2,3


In [8]:
misty_data

,Timestamp,Description,Interst/Alarm,Confusion/Understanding,Frusteration/Relief,Sorrow/Joy,Anger/Gratitude,Fear/Hope,Boredom/Surprise,Disgust/Desire
1,1.617916e+09,confused and asking for clarification almost,3,2,4,3,2,4,1,1
2,1.619713e+09,"His eyes half closed, his head tilted up, his ...",2,4,3,3,4,4,3,5
3,1.619713e+09,"eyes were half open, head tilted up and down s...",3,3,2,3,3,3,1,3
4,1.619713e+09,"eyes were looking concerned to the left, then ...",3,2,3,3,3,3,4,3
5,1.619713e+09,one arm moved about halfway up twice head tilt...,3,3,2,2,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...
636,1.637353e+09,The robot moved it head up and down a few time...,2,4,5,5,5,5,4,5
638,1.637353e+09,"The robot said ""um"" and raised its eyebrow and...",4,1,1,3,1,2,5,2
639,1.637353e+09,Misty raised her hed up and down a few times. ...,5,1,2,3,3,1,5,3
641,1.637354e+09,"At first, the robot said ""huh"" and looked up a...",4,2,2,3,3,3,4,4


In [9]:
MAX_WORDS = 80
pad = lambda x: np.pad(x, [[0,MAX_WORDS - len(x[:MAX_WORDS])], [0,0]], 'constant')
mask = np.random.rand(3072) > 0

def get_embedding(sent):
    if(len(sent.strip()) == 0):
        return np.zeros((MAX_WORDS, 3072))
    sentence = Sentence(sent)
    BERT.embed(sentence)
    embed = pad(np.array([w.embedding.cpu().numpy()[mask] for w in sentence]))
    return embed

In [10]:
cozmo_data['Description_Bert'] = cozmo_data['Description'].apply(get_embedding)

In [11]:
cozmo_data

,Timestamp,Description,Interst/Alarm,Confusion/Understanding,Frusteration/Relief,Sorrow/Joy,Anger/Gratitude,Fear/Hope,Boredom/Surprise,Disgust/Desire,Description_Bert
0,1.617917e+09,It went to investigate a thing it saw,1,1,3,3,3,4,4,5,"[[0.003090775, -0.33126733, -0.42522523, 0.122..."
1,1.619714e+09,drove forward for a bit and then turned to his...,2,3,3,3,3,3,3,3,"[[0.41755724, -0.010028644, 0.1145585, -0.2166..."
2,1.619714e+09,"just drove forward, made a little sound at the...",2,3,3,3,3,3,3,3,"[[0.38736537, -0.13056435, -0.22778009, 0.6008..."
4,1.619714e+09,looked down squinted eyes and then looked forw...,2,2,3,3,3,3,3,3,"[[0.38436273, 0.5813796, -0.046782747, -0.2924..."
5,1.619714e+09,"lifted front part up, moved forward and turned...",4,3,3,3,2,3,4,3,"[[0.265327, 0.18869174, 0.9075626, -0.40546942..."
...,...,...,...,...,...,...,...,...,...,...,...
648,1.637355e+09,At first the robot tilted thier head up and sa...,1,2,3,1,3,3,3,3,"[[-0.03434295, 0.5708942, -0.37369967, -0.1584..."
649,1.637355e+09,The robot first moved straight with a slight r...,1,2,1,3,1,2,3,1,"[[-0.11550679, -0.27403948, -0.63136435, 0.280..."
650,1.637355e+09,"At first, the robot raised their head and hand...",1,5,5,5,5,5,5,3,"[[0.20224547, 0.4459168, -0.37732568, -0.21969..."
651,1.637355e+09,"The robot turned around nearly 180 degrees, lo...",3,4,3,1,3,1,2,3,"[[0.34809294, 0.30013406, -0.654603, -0.123494..."


In [12]:
X = np.stack(cozmo_data.Description_Bert.values, axis=0)

# If we want to concat the word vectors:
# X = X.reshape(X.shape[:-2] + (-1,))

# If we want to sum the word vectors:
X = np.sum(X, axis=1)

Y = cozmo_data.loc[:, 'Interst/Alarm':'Disgust/Desire'].values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
X.shape, Y.shape, X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((547, 3072), (547, 8), (492, 3072), (55, 3072), (492, 8), (55, 8))

In [16]:
model = Sequential()
model.add(Dense(1, activation='relu'))
model.add(Dense(8, activation='relu'))
model.compile(loss='kl_divergence', optimizer='adam', metrics=['accuracy'])

epochs = 800
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=1, batch_size=492)
model.summary()

Epoch 1/800
1/1 [==============================] - 31s 31s/step - loss: 89.6041 - accuracy: 0.2602 - val_loss: 50.9314 - val_accuracy: 0.1636
Epoch 2/800
1/1 [==============================] - 0s 38ms/step - loss: 50.5423 - accuracy: 0.2033 - val_loss: 48.6098 - val_accuracy: 0.1636
Epoch 3/800
1/1 [==============================] - 0s 31ms/step - loss: 48.6196 - accuracy: 0.2134 - val_loss: 48.4731 - val_accuracy: 0.1636
Epoch 4/800
1/1 [==============================] - 0s 34ms/step - loss: 48.4564 - accuracy: 0.2134 - val_loss: 48.4265 - val_accuracy: 0.1636
Epoch 5/800
1/1 [==============================] - 0s 33ms/step - loss: 48.4129 - accuracy: 0.2134 - val_loss: 48.4051 - val_accuracy: 0.1636
Epoch 6/800
1/1 [==============================] - 0s 40ms/step - loss: 48.3934 - accuracy: 0.2134 - val_loss: 48.3923 - val_accuracy: 0.1636
Epoch 7/800
1/1 [==============================] - 0s 58ms/step - loss: 48.3838 - accuracy: 0.2134 - val_loss: 48.3846 - val_accuracy: 0.1636
Epoch 